In [1]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None
import re
from tqdm import tqdm
from IPython.display import Image, display
from datetime import date, timedelta
import numpy as np
from matplotlib.pyplot import *
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

def build_area_seqs(target_area, start='2016-08-01', end='2017-01-01'):
    # 整合到一个文件中
    area_df = pd.DataFrame()
    for name in target_area.parking_name:
        file_name = 'generated/data/seqs/'+name+'_seq.csv'
        file_df = pd.read_csv(file_name)
        file_df['parking'] = nks[name]
        cols = file_df.columns.tolist()
        cols = [cols[0], cols[2], cols[1]]
        file_df = file_df[cols]
        if len(area_df)>0:
            area_df = pd.concat([area_df, file_df])
        else:
            area_df = file_df

    out_bound_indexes = area_df[(area_df['date'] < start) | (area_df['date'] >= end)].index 
    area_df.drop(out_bound_indexes, inplace = True) 
    return area_df.pivot_table('occupy', ['date'], 'parking')


def get_nodes_features(area_df):
    node_f = area_df[['total_space','monthly_fee','building_type']]
    node_f.loc[:,['total_space', 'monthly_fee']] = min_max_scaler.fit_transform(node_f[['total_space', 'monthly_fee']])
    building_type_oneHot = pd.get_dummies(node_f['building_type'])
    node_f = node_f.drop('building_type',axis = 1)
    node_f = node_f.join(building_type_oneHot)
    return node_f

def max_min_scale(raw):
    x_scaled = min_max_scaler.fit_transform(raw.numpy())
    return pd.DataFrame(x_scaled)

In [2]:
basic_info_df = pd.read_csv('generated/data/parkings_info.csv')
basic_info_df['lat_long'] = list(zip(basic_info_df['latitude'], basic_info_df['longitude']))

from graph_utils import build_graph

target_park = '大信大厦'
use_gcn=False
graph_nodes_max_dis = 0.5 if use_gcn else 0.1

target_area, adj, target_map, nks, kns = build_graph(basic_info_df, target_park, max_dis=graph_nodes_max_dis)
target_park_basic_info = basic_info_df.loc[basic_info_df.parking_name == target_park].iloc[0]
key = nks[target_park]
node_f = get_nodes_features(target_area)
seqs_raw = build_area_seqs(target_area , start='2016-06-02', end='2016-08-04')
seqs_normal = seqs_raw.copy().fillna(0)
seqs_normal[seqs_normal.columns.values] = min_max_scaler.fit_transform(seqs_normal[seqs_normal.columns.values])

In [3]:
import tensorflow as tf
from abc import ABC
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, GRU, Flatten, Dense, LeakyReLU
from spektral.layers import GraphConv

l2_reg = 5e-4 / 2  # L2 regularization rate


class Generator(Model, ABC):

    def __init__(self, adj, nodes_features, base=8, dropout=0.5):
        super(Generator, self).__init__()
        self.adj = adj
        self.nodes_features = nodes_features

        self.dropout = Dropout(dropout)
        self.flatten = Flatten()
        self.graph_conv_1 = GraphConv(128,
                                      activation='elu',
                                      kernel_regularizer=l2(l2_reg),
                                      use_bias=False)
        self.graph_conv_2 = GraphConv(64,
                                      activation='elu',
                                      kernel_regularizer=l2(l2_reg),
                                      use_bias=False)
        self.dense_1 = Dense(256, activation='relu')
        self.dense_2 = Dense(512, activation='relu')
        self.gru = GRU(128, return_sequences=True)
        self.final_dense = Dense(1, activation='tanh')

    def call(self, seq, training=True):
        s = tf.convert_to_tensor(seq)  # S*N
        
        if use_gcn:
            f = tf.convert_to_tensor(self.nodes_features)  # N*F
            g = tf.convert_to_tensor(self.adj)  # N*N
            c = self.graph_conv_1([f, g])  # N*Cov1
            c = self.graph_conv_2([c, g])  # N*Cov2
            s = tf.matmul(s, c)  # S*N x N*Cov2

        fc = self.dense_1(s)  # S*D1
        fc = self.dropout(fc, training=training)
        fc = self.dense_2(fc)  # S*D2
        fc = self.dropout(fc, training=training)

        fc = tf.expand_dims(fc, axis=0) 
        ro = self.gru(fc)
        ro = tf.squeeze(ro, axis=0)  # S*R
        return self.final_dense(ro)  # S*1


class Discriminator(Model, ABC):

    def __init__(self, adj, nodes_features, base=8, dropout=0.5, alpha=0.2):
        super(Discriminator, self).__init__()
        self.adj = adj
        self.nodes_features = nodes_features

        self.leaky_relu = LeakyReLU(alpha=alpha)
        self.dropout = Dropout(dropout)
        self.flatten = Flatten()
        self.graph_conv_1 = GraphConv(128,
                                      activation='elu',
                                      kernel_regularizer=l2(l2_reg),
                                      use_bias=False)
        self.graph_conv_2 = GraphConv(64,
                                      activation='elu',
                                      kernel_regularizer=l2(l2_reg),
                                      use_bias=False)
        self.dense_1 = Dense(256)
        self.dense_2 = Dense(512)
        self.gru = GRU(128, return_sequences=True)
        self.final_dense = Dense(1, activation='sigmoid')

    def call(self, seq, training=True):
        s = tf.convert_to_tensor(seq)  # S*N
        
        if use_gcn:
            f = tf.convert_to_tensor(self.nodes_features)  # N*F
            g = tf.convert_to_tensor(self.adj)  # N*N
            c = self.graph_conv_1([f, g])  # N*Cov1
            c = self.graph_conv_2([c, g])  # N*Cov2
            s = tf.matmul(s, c)  # S*N x N*Cov2
        
        fc = self.dense_1(s)  # S*D1
        fc = self.leaky_relu(fc)
        fc = self.dropout(fc, training=training)
        fc = self.dense_2(fc)  # S*D2
        fc = self.leaky_relu(fc)
        fc = self.dropout(fc, training=training)

        fc = tf.expand_dims(fc, axis=0)
        ro = self.gru(fc)
        ro = tf.squeeze(ro, axis=0)  # S*R
        return self.final_dense(ro)  # S*1


INFO:tensorflow:Using local port 21193
INFO:tensorflow:Using local port 17282
INFO:tensorflow:Using local port 23957
INFO:tensorflow:Using local port 16997
INFO:tensorflow:Using local port 18490
INFO:tensorflow:Using local port 21323
INFO:tensorflow:Using local port 15833
INFO:tensorflow:Using local port 15335
INFO:tensorflow:Using local port 23242
INFO:tensorflow:Using local port 16635


In [4]:
import os
from tensorflow.keras.optimizers import Adam
from spektral.utils import normalized_laplacian
# from model import Generator, Discriminator
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

save_week_interval = 10
save_all_interval = 100

class Train:
    def __init__(self, seqs, adj, nodes_features, epochs=1000,
                 lr = 1e-3, adam_beta_1 = 0.5, base=8, dropout=0.5, alpha=0.2):
        self.epochs = epochs
        self.seqs = seqs.astype('float32')

        self.gen_optimizer = Adam(lr, adam_beta_1)
        self.desc_optimizer = Adam(lr, adam_beta_1)

        self.adj = normalized_laplacian(adj.astype('float32'))
        self.nodes_features = nodes_features.astype('float32')
        self.generator = Generator(self.adj, self.nodes_features, base, dropout)
        self.discriminator = Discriminator(self.adj, self.nodes_features, base, dropout, alpha)
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    def __call__(self, epochs=None ,save_path='generated/'):
        save_path += str(time.time())
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        if epochs is None:
            epochs = self.epochs
            
        time_len = self.seqs.shape[0]
        num_nodes = self.seqs.shape[1]
        total_batch = int(time_len / batch_size)  # 2976/96=31

        time_consumed_total = 0.
        for epoch in range(1, epochs + 1):
            start = time.time()
            total_gen_loss = 0
            total_disc_loss = 0

            for week in range(0, total_batch):
                current_seqs = self.seqs[week*batch_size:week*batch_size + batch_size]
                seqs_noised = current_seqs.copy()
                max_s = current_seqs[key].max()
                seqs_noised[key] = np.random.normal(max_s / 2.0, max_s / 10.0,
                                                         size=(current_seqs.shape[0])).astype('float32')
                gen_loss, disc_loss = self.train_step(current_seqs, seqs_noised)
                total_gen_loss += gen_loss
                total_disc_loss += disc_loss

            time_consumed = time.time() - start
            time_consumed_total += time_consumed
            time_consumed_agv = time_consumed_total / epoch
            epochs_last = epochs - epoch
            estimate_time_last = epochs_last * time_consumed_agv
            print('epoch {}({})/{}({}) - gen_loss = {}, disc_loss = {}, estimated to finish: {}'
              .format(epoch, round(time.time() - start, 2),
                      epochs, round(time_consumed_total, 2),
                      round(float(total_gen_loss / total_batch), 5),
                      round(float(total_disc_loss / total_batch), 5),
                      round(estimate_time_last, 2)))
            
            if epoch % save_week_interval == 0:
                self.compare_plot('week_'+str(epoch), save_path, int(total_batch/2)*7)
            if epoch % save_all_interval == 0:
                self.compare_plot('all_'+str(epochs), save_path, 0, total_batch) 

    @tf.function
    def train_step(self, seqs, seqs_noised):
        with tf.GradientTape(persistent=True) as tape:
            real_output = self.discriminator(seqs)  # 评价高
            generated = self.generator(seqs_noised)
            left = tf.slice(seqs, [0, 0], [batch_size, key])
            right = tf.slice(seqs, [0, key + 1], [batch_size, -1])
            combined = tf.concat([left, generated, right], 1)
            generated_output = self.discriminator(combined)  # 初始评价低

            loss_g = self.generator_loss(self.cross_entropy, generated_output)
            loss_d = self.discriminator_loss(self.cross_entropy, real_output, generated_output)

        grad_gen = tape.gradient(loss_g, self.generator.trainable_variables)
        grad_disc = tape.gradient(loss_d, self.discriminator.trainable_variables)

        self.gen_optimizer.apply_gradients(zip(grad_gen, self.generator.trainable_variables))
        self.desc_optimizer.apply_gradients(zip(grad_disc, self.discriminator.trainable_variables))

        return loss_g, loss_d

    def generate_first(self):
        import random as random
        week = 1 # random.randint(0,31)
        seqs_replace = self.seqs[week*batch_size:week*batch_size + batch_size].copy()
        max_s = seqs_replace[key].max()
        seqs_replace[key] = np.random.normal(max_s / 2.0, max_s / 10.0, size=(seqs_replace.shape[0])).astype(
            'float32')
        gen_data = self.generator(seqs_replace, training=False)
        return max_min_scale(gen_data)

    def generate(self, real_seqs):
        seqs_replace = real_seqs.copy()
        max_s = seqs_replace[key].max()
        seqs_replace[key] = np.random.normal(max_s / 2.0, max_s / 10.0, size=(seqs_replace.shape[0])).astype(
            'float32')
        gen_data = self.generator(seqs_replace, training=False)
        return pd.DataFrame(gen_data.numpy())
    
    @staticmethod
    def discriminator_loss(loss_object, real_output, fake_output):
        real_loss = loss_object(tf.ones_like(real_output), real_output)
        fake_loss = loss_object(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    @staticmethod
    def generator_loss(loss_object, fake_output):
        return loss_object(tf.ones_like(fake_output), fake_output)
    
    def compare_plot(self, name, save_path, start_day=0, week=1):
        fig, ax = subplots()
        fig.set_figheight(8)
        fig.set_figwidth(20)
        real_seqs = self.seqs[start_day:start_day+batch_size*week]
        generated_seqs = []
        for w in range(week):
            generated_seq = self.generate(real_seqs[start_day*w:start_day*w+batch_size])
            if len(generated_seqs) == 0:
                generated_seqs = generated_seq
            else:
                generated_seqs =  generated_seqs.append(generated_seq, ignore_index=True)
        all_seqs = pd.concat([pd.DataFrame(real_seqs[key].values), generated_seqs], axis=1)
        all_seqs.plot(ax=ax)
        n=2
        ax.legend(['real'+str(w) for w in range(1,n)]+['gen'+str(w) for w in range(1,n)]);
        fig.savefig(save_path+"/compare_" + name + ".png")
        plt.close()

In [5]:
batch_size = 96*7

lr = 0.0001
adm1 = 0.5
base = 8
epochs=1200
dropout = 0.3
alpha = 0.2
name='GCN_'+str(use_gcn)+'_'+str(lr)+'_'+str(adm1)+'_'+str(dropout)+'_'+str(alpha)
save_path='generated/'+str(epochs)

In [6]:
train = Train(seqs_normal, adj, node_f, epochs, lr = lr, adam_beta_1 = adm1, base=base, dropout=dropout, alpha=alpha)
train(epochs, save_path+'/'+name+'_')

C:\Users\marko\anaconda3\lib\site-packages\spektral\utils\convolution.py:30: RuntimeWarning: divide by zero encountered in power
  degrees = np.power(np.array(A.sum(1)), k).flatten()


epoch 1(7.8)/1200(7.8) - gen_loss = 0.68714, disc_loss = 1.35392, estimated to finish: 9349.77
epoch 2(4.24)/1200(12.04) - gen_loss = 0.63602, disc_loss = 1.35221, estimated to finish: 7213.14
epoch 3(3.92)/1200(15.96) - gen_loss = 0.61399, disc_loss = 1.41618, estimated to finish: 6368.83
epoch 4(3.92)/1200(19.89) - gen_loss = 0.71979, disc_loss = 1.40191, estimated to finish: 5945.9
epoch 5(3.95)/1200(23.84) - gen_loss = 0.82296, disc_loss = 1.37186, estimated to finish: 5696.68
epoch 6(4.18)/1200(28.02) - gen_loss = 0.8119, disc_loss = 1.38248, estimated to finish: 5575.48
epoch 7(4.29)/1200(32.31) - gen_loss = 0.72334, disc_loss = 1.39821, estimated to finish: 5506.8
epoch 8(4.01)/1200(36.32) - gen_loss = 0.6733, disc_loss = 1.37784, estimated to finish: 5412.2
epoch 9(4.14)/1200(40.47) - gen_loss = 0.64093, disc_loss = 1.38162, estimated to finish: 5354.94
epoch 10(4.12)/1200(44.58) - gen_loss = 0.66186, disc_loss = 1.39756, estimated to finish: 5305.44
epoch 11(4.15)/1200(48.74) 

epoch 84(4.38)/1200(353.02) - gen_loss = 0.67076, disc_loss = 1.40625, estimated to finish: 4690.1
epoch 85(4.68)/1200(357.7) - gen_loss = 0.6724, disc_loss = 1.37994, estimated to finish: 4692.14
epoch 86(4.74)/1200(362.44) - gen_loss = 0.67729, disc_loss = 1.35233, estimated to finish: 4694.83
epoch 87(4.48)/1200(366.92) - gen_loss = 0.68033, disc_loss = 1.32127, estimated to finish: 4694.06
epoch 88(4.47)/1200(371.4) - gen_loss = 0.65926, disc_loss = 1.31659, estimated to finish: 4693.1
epoch 89(4.77)/1200(376.16) - gen_loss = 0.59289, disc_loss = 1.41081, estimated to finish: 4695.71
epoch 90(4.39)/1200(380.55) - gen_loss = 0.60338, disc_loss = 1.46311, estimated to finish: 4693.45
epoch 91(4.23)/1200(384.78) - gen_loss = 0.68207, disc_loss = 1.42168, estimated to finish: 4689.19
epoch 92(4.58)/1200(389.36) - gen_loss = 0.76799, disc_loss = 1.37068, estimated to finish: 4689.23
epoch 93(4.56)/1200(393.92) - gen_loss = 0.85084, disc_loss = 1.32678, estimated to finish: 4688.94
epoch

epoch 166(3.86)/1200(676.0) - gen_loss = 0.68635, disc_loss = 1.38759, estimated to finish: 4210.77
epoch 167(3.82)/1200(679.83) - gen_loss = 0.69558, disc_loss = 1.38127, estimated to finish: 4205.15
epoch 168(3.83)/1200(683.65) - gen_loss = 0.70316, disc_loss = 1.37699, estimated to finish: 4199.59
epoch 169(3.89)/1200(687.54) - gen_loss = 0.70666, disc_loss = 1.37602, estimated to finish: 4194.43
epoch 170(3.91)/1200(691.46) - gen_loss = 0.70457, disc_loss = 1.37832, estimated to finish: 4189.42
epoch 171(3.83)/1200(695.28) - gen_loss = 0.69734, disc_loss = 1.38371, estimated to finish: 4183.91
epoch 172(3.87)/1200(699.16) - gen_loss = 0.68962, disc_loss = 1.38779, estimated to finish: 4178.68
epoch 173(3.83)/1200(702.99) - gen_loss = 0.68369, disc_loss = 1.39041, estimated to finish: 4173.22
epoch 174(3.88)/1200(706.86) - gen_loss = 0.68134, disc_loss = 1.38959, estimated to finish: 4168.04
epoch 175(3.81)/1200(710.67) - gen_loss = 0.6823, disc_loss = 1.38584, estimated to finish: 

epoch 248(3.91)/1200(991.78) - gen_loss = 1.09234, disc_loss = 1.39373, estimated to finish: 3807.16
epoch 249(3.81)/1200(995.59) - gen_loss = 1.0445, disc_loss = 1.26438, estimated to finish: 3802.44
epoch 250(3.89)/1200(999.48) - gen_loss = 0.98158, disc_loss = 1.03022, estimated to finish: 3798.03
epoch 251(3.8)/1200(1003.29) - gen_loss = 0.96537, disc_loss = 0.79853, estimated to finish: 3793.3
epoch 252(3.87)/1200(1007.16) - gen_loss = 0.99613, disc_loss = 0.62466, estimated to finish: 3788.83
epoch 253(3.8)/1200(1010.95) - gen_loss = 1.03632, disc_loss = 0.60562, estimated to finish: 3784.08
epoch 254(3.83)/1200(1014.78) - gen_loss = 1.06474, disc_loss = 0.59641, estimated to finish: 3779.47
epoch 255(3.85)/1200(1018.64) - gen_loss = 1.07338, disc_loss = 0.60294, estimated to finish: 3774.95
epoch 256(3.81)/1200(1022.45) - gen_loss = 0.39346, disc_loss = 2.85406, estimated to finish: 3770.27
epoch 257(3.86)/1200(1026.31) - gen_loss = 0.05304, disc_loss = 3.11382, estimated to fin

...

epoch 815(3.81)/1200(3171.66) - gen_loss = 4.06379, disc_loss = 0.19641, estimated to finish: 1498.27
epoch 816(3.78)/1200(3175.44) - gen_loss = 4.06788, disc_loss = 0.2057, estimated to finish: 1494.33
epoch 817(3.81)/1200(3179.25) - gen_loss = 4.12733, disc_loss = 0.20865, estimated to finish: 1490.4
epoch 818(3.78)/1200(3183.03) - gen_loss = 4.1087, disc_loss = 0.20261, estimated to finish: 1486.45
epoch 819(3.8)/1200(3186.83) - gen_loss = 4.19053, disc_loss = 0.18709, estimated to finish: 1482.52
epoch 820(3.81)/1200(3190.65) - gen_loss = 4.23734, disc_loss = 0.18094, estimated to finish: 1478.59
epoch 821(3.79)/1200(3194.44) - gen_loss = 4.27457, disc_loss = 0.17927, estimated to finish: 1474.65
epoch 822(4.08)/1200(3198.52) - gen_loss = 4.22772, disc_loss = 0.19216, estimated to finish: 1470.85
epoch 823(4.2)/1200(3202.72) - gen_loss = 4.31696, disc_loss = 0.18629, estimated to finish: 1467.1
epoch 824(4.52)/1200(3207.24) - gen_loss = 4.33709, disc_loss = 0.17508, estimated to fi

epoch 896(3.76)/1200(3481.44) - gen_loss = 4.74321, disc_loss = 0.29253, estimated to finish: 1181.2
epoch 897(3.81)/1200(3485.25) - gen_loss = 4.63429, disc_loss = 0.36773, estimated to finish: 1177.29
epoch 898(3.83)/1200(3489.08) - gen_loss = 4.29864, disc_loss = 0.29219, estimated to finish: 1173.39
epoch 899(3.8)/1200(3492.88) - gen_loss = 4.78478, disc_loss = 0.29372, estimated to finish: 1169.47
epoch 900(3.86)/1200(3496.74) - gen_loss = 4.65717, disc_loss = 0.28921, estimated to finish: 1165.58
epoch 901(3.77)/1200(3500.51) - gen_loss = 4.55785, disc_loss = 0.31096, estimated to finish: 1161.66
epoch 902(3.82)/1200(3504.33) - gen_loss = 4.67681, disc_loss = 0.28881, estimated to finish: 1157.75
epoch 903(3.77)/1200(3508.11) - gen_loss = 4.61426, disc_loss = 0.33636, estimated to finish: 1153.83
epoch 904(3.8)/1200(3511.9) - gen_loss = 4.65084, disc_loss = 0.34209, estimated to finish: 1149.92
epoch 905(3.8)/1200(3515.71) - gen_loss = 4.64547, disc_loss = 0.36226, estimated to f

epoch 978(3.76)/1200(3793.58) - gen_loss = 5.45828, disc_loss = 0.19194, estimated to finish: 861.12
epoch 979(3.77)/1200(3797.35) - gen_loss = 5.44005, disc_loss = 0.27993, estimated to finish: 857.22
epoch 980(3.76)/1200(3801.11) - gen_loss = 3.2695, disc_loss = 0.80567, estimated to finish: 853.31
epoch 981(3.77)/1200(3804.88) - gen_loss = 3.94835, disc_loss = 0.35512, estimated to finish: 849.41
epoch 982(3.77)/1200(3808.66) - gen_loss = 4.61467, disc_loss = 0.24403, estimated to finish: 845.51
epoch 983(3.77)/1200(3812.43) - gen_loss = 5.21281, disc_loss = 0.22555, estimated to finish: 841.6
epoch 984(3.8)/1200(3816.23) - gen_loss = 5.31827, disc_loss = 0.22488, estimated to finish: 837.71
epoch 985(3.78)/1200(3820.01) - gen_loss = 5.40534, disc_loss = 0.22883, estimated to finish: 833.81
epoch 986(3.83)/1200(3823.84) - gen_loss = 5.31996, disc_loss = 0.22634, estimated to finish: 829.92
epoch 987(3.8)/1200(3827.64) - gen_loss = 5.3143, disc_loss = 0.22605, estimated to finish: 82

epoch 1059(3.81)/1200(4105.35) - gen_loss = 0.7595, disc_loss = 1.28466, estimated to finish: 546.6
epoch 1060(3.75)/1200(4109.1) - gen_loss = 0.62121, disc_loss = 1.40247, estimated to finish: 542.71
epoch 1061(3.83)/1200(4112.93) - gen_loss = 0.73315, disc_loss = 1.29372, estimated to finish: 538.83
epoch 1062(3.82)/1200(4116.75) - gen_loss = 0.89015, disc_loss = 1.16321, estimated to finish: 534.94
epoch 1063(3.79)/1200(4120.54) - gen_loss = 0.95381, disc_loss = 1.0984, estimated to finish: 531.06
epoch 1064(3.84)/1200(4124.38) - gen_loss = 0.73631, disc_loss = 1.2781, estimated to finish: 527.18
epoch 1065(3.8)/1200(4128.18) - gen_loss = 0.52332, disc_loss = 1.51361, estimated to finish: 523.29
epoch 1066(3.82)/1200(4131.99) - gen_loss = 0.65331, disc_loss = 1.34894, estimated to finish: 519.41
epoch 1067(3.81)/1200(4135.8) - gen_loss = 0.73828, disc_loss = 1.27036, estimated to finish: 515.52
epoch 1068(3.76)/1200(4139.56) - gen_loss = 0.7872, disc_loss = 1.22554, estimated to fin

epoch 1140(3.99)/1200(4417.71) - gen_loss = 3.367, disc_loss = 0.52356, estimated to finish: 232.51
epoch 1141(3.98)/1200(4421.69) - gen_loss = 3.37378, disc_loss = 0.50572, estimated to finish: 228.64
epoch 1142(4.07)/1200(4425.76) - gen_loss = 3.39215, disc_loss = 0.48499, estimated to finish: 224.78
epoch 1143(4.08)/1200(4429.84) - gen_loss = 3.34652, disc_loss = 0.49783, estimated to finish: 220.91
epoch 1144(4.18)/1200(4434.02) - gen_loss = 3.35196, disc_loss = 0.46589, estimated to finish: 217.05
epoch 1145(4.14)/1200(4438.17) - gen_loss = 3.36357, disc_loss = 0.48452, estimated to finish: 213.19
epoch 1146(4.25)/1200(4442.42) - gen_loss = 3.35041, disc_loss = 0.46668, estimated to finish: 209.33
epoch 1147(4.18)/1200(4446.6) - gen_loss = 3.39125, disc_loss = 0.45573, estimated to finish: 205.47
epoch 1148(3.89)/1200(4450.49) - gen_loss = 3.34995, disc_loss = 0.47124, estimated to finish: 201.59
epoch 1149(3.84)/1200(4454.33) - gen_loss = 3.38257, disc_loss = 0.4468, estimated to